artist.json.gzは，オープンな音楽データベースMusicBrainzの中で，アーティストに関するものをJSON形式に変換し，gzip形式で圧縮したファイルである．

このファイルには，1アーティストに関する情報が1行にJSON形式で格納されている．

JSON形式の概要は以下の通りである．

artist.json.gzのデータをKey-Value-Store (KVS) およびドキュメント志向型データベースに格納・検索することを考える．

KVSとしては，LevelDB，Redis，KyotoCabinet等を用いよ．

ドキュメント志向型データベースとして，MongoDBを採用したが，CouchDBやRethinkDB等を用いてもよい．

** 0. KVSの構築 **

Key-Value-Store (KVS) を用い，アーティスト名（name）から活動場所（area）を検索するためのデータベースを構築せよ．

In [1]:
import gzip
import json

*** leveldbパッケージを使った場合 ***

In [2]:
fname = 'nlp100data/artist.json.gz'
fname_db = 'nlp100data/test_db'

In [3]:
import leveldb

In [4]:
db = leveldb.LevelDB(fname_db)

In [5]:
with gzip.open(fname, 'rt') as data_file:
    for i , line in enumerate(data_file):
        if i > -1:
            run_dict = json.loads(line)
            key = "{}\t{}".format(run_dict['name'] , run_dict['id'])
            #print(key)
            value = run_dict.get("area" , "")
            db.Put(key.encode() , value.encode())

            #print("[{}] {}".format(value ,key))

In [7]:
Ndata = len(list(db.RangeIter(include_value=False)))

In [8]:
print("registered {} pairs".format(Ndata))

registered 921337 pairs


**** sandbox ****


In [6]:
run_dict

{'aliases': [{'name': 'Ranks, Shabba', 'sort_name': 'Ranks, Shabba'},
  {'name': 'Shaba Ranks', 'sort_name': 'Shaba Ranks'},
  {'name': 'Snabba Ranks', 'sort_name': 'Snabba Ranks'},
  {'name': 'Shabba', 'sort_name': 'Shabba'}],
 'area': 'Jamaica',
 'begin': {'date': 17, 'month': 1, 'year': 1966},
 'ended': True,
 'gender': 'Male',
 'gid': 'c0336f2a-610f-4b38-bc13-eb07393425a7',
 'id': 37082,
 'name': 'Shabba Ranks',
 'sort_name': 'Ranks, Shabba',
 'tags': [{'count': 1, 'value': 'raggamuffin'},
  {'count': 1, 'value': 'dancehall'},
  {'count': 1, 'value': 'pop'},
  {'count': 1, 'value': 'pop and chart'},
  {'count': 1, 'value': 'drum and bass'},
  {'count': 1, 'value': 'jungle'},
  {'count': 1, 'value': 'mc'}],
 'type': 'Person'}

In [11]:
%%bash
pwd

/home/toshinao/PycharmProjects/keras_sandbox


In [17]:
import codecs

tmp = list()
with codecs.open('nlp100data/tmp' , 'r' , 'utf-8') as rf:
    for l in rf:
        tmp.append(l)

In [16]:
tmp[0]

'{"name": "WIK▲N", "tags": [{"count": 1, "value": "sillyname"}], "sort_name": "WIK▲N", "ended": true, "gid": "8972b1c1-6482-4750-b51f-596d2edea8b1", "id": 805192}\n'


*** plyvelパッケージを使った場合 (暫定的にこちらをメインに)***

https://qiita.com/tomotaka_ito/items/60c65dd5261fdfc6e71a

https://plyvel.readthedocs.io/en/latest/user.html#getting-started

http://kuroneko0208.hatenablog.com/entry/2014/10/06/090128

In [2]:
fname = 'nlp100data/artist.json.gz'
fname_db = 'nlp100data/test2_db'

In [3]:
import plyvel

In [4]:
plyvel.DB?

In [5]:
db = plyvel.DB(fname_db , create_if_missing=True)

In [6]:
with gzip.open(fname, 'rt') as data_file:
    for i , line in enumerate(data_file):
        if i > -1:
            run_dict = json.loads(line)
            #key = "{}\t{}".format(run_dict['name'] , run_dict['id'])
            key = run_dict['name']
            #print(key)
            value = run_dict.get("area" , "")
            if value != "":
                db.put(key.encode() , value.encode())

            #print("[{}] {}".format(value ,key))

In [7]:
len([key for key , value in db])

309317

In [8]:
db.close()

#### KVSの検索

60で構築したデータベースを用い，特定の（指定された）アーティストの活動場所を取得せよ．

In [9]:
db = plyvel.DB(fname_db , create_if_missing=True)

In [25]:
artist_name = "10 Rue d'la Madeleine"

In [26]:
db.get(artist_name.encode()).decode()

'France'

##### sandbox

In [16]:
tmp = [key_value[0].decode()  for i , key_value in enumerate(db) if i < 300 and i > 290]

In [17]:
tmp

['1/3 Octave Band',
 '10 5 Neuton',
 '10 Fold B-Low',
 '10 Ft. Ganja Plant',
 '10 Minute Warning',
 "10 Rue d'la Madeleine",
 '10 Years',
 '10 petits indiens',
 '10" Maria']

In [24]:
db.get("10 Rue d'la Madeleine".encode()).decode()

'France'

In [35]:
db.get('"Pretty" Fedd'.encode())

b''

In [32]:
["{} [{}]".format(key_value[0].decode() , key_value[1].decode()) for i , key_value in enumerate(db) if i < 100 and i > 90]

['"Pretty" Fedd []',
 '"Red" Roberts mit seinem Ultraphon Jazz-Orchester [Germany]',
 '"Ritekt Jerk", Mases Erik Jonsson []',
 '"Rob,SEAL" []',
 '"Rolling Joe" Johnson []',
 '"Rural" Merle Hicks []',
 '"See There" Singing Band []',
 '"Sir" Oliver Mally [Austria]',
 '"Slim" Jim Smith []']

In [12]:
sample_names = [key_value[0] for i , key_value in enumerate(db) if i < 100]

In [13]:
sample_names[70].decode()

"'Jabbo' Williams"

In [15]:
db.get(sample_names[7]).decode()

'Sankt-Peterburg'

In [25]:
db.get(b"!ATTENTION!\t617688")

b''

In [26]:
db.get(b'!Action Pact!\\t95246')

In [28]:
db.get(b'!Bang Elektronika\\t172294')

In [41]:
%%bash
git commit -a -m "60"

[master 6618d1c] 60
 1 file changed, 128 insertions(+), 8 deletions(-)


*** バイト列型に関するメモ ***

#### KVS内の反復処理
60で構築したデータベースを用い，活動場所が「Japan」となっているアーティスト数を求めよ．

In [32]:

run_place = "Japan"

In [33]:
run_place.encode()

b'Japan'

In [42]:
japan_artists = [x[0].decode() for x in db.iterator() if x[1] == run_place.encode()]

In [44]:
print("Number of registered artists in Japan is {}".format(len(japan_artists)))

Number of registered artists in Japan is 22128


In [49]:
japan_artists[100:120]

['6号さん',
 '7!!',
 '72',
 '742',
 '765PRO ALLSTARS',
 '7@',
 '7chi子♪',
 '7人祭',
 '800 Cherries',
 '876PRO ALLSTARS',
 '96',
 '96ちゃん',
 '98',
 '99RadioService',
 '9GOATS BLACK OUT',
 '9mm Parabellum Bullet',
 '9nine',
 '9少女',
 '>>96',
 'A Lunch']

####  オブジェクトを値に格納したKVS
KVSを用い，アーティスト名（name）からタグと被タグ数（タグ付けされた回数）のリストを検索するためのデータベースを構築せよ．さらに，ここで構築したデータベースを用い，アーティスト名からタグと被タグ数を検索せよ．

In [52]:
fname_db63 = 'nlp100data/test63_db'

In [53]:
db = plyvel.DB(fname_db63 , create_if_missing=True)

In [57]:
with gzip.open(fname, 'rt') as data_file:
    for i , line in enumerate(data_file):
        if i > -1:
            run_dict = json.loads(line)
            #key = "{}\t{}".format(run_dict['name'] , run_dict['id'])
            key = run_dict['name']
            #print(key)
            value = run_dict.get("tags" , "")
            if value != "":
                db.put(key.encode() , json.dumps(value).encode())

            #print("[{}] {}".format(value ,key))

In [56]:
json.dumps(value)

'[{"value": "sillyname", "count": 1}]'

In [47]:
%%bash
git commit -a -m'from notebook'

[master 095489d] from notebook
 1 file changed, 393 insertions(+), 45 deletions(-)
